In [1]:
# LEFT OFF:  Need to split the models.  1 for items with age of well.  1 without.

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV

%matplotlib inline

c:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [3]:
# mac
# workdir = '/Project/data/dd_WaterTable/'
# windows
workdir = 'C:\\project\\data\\dd_watertable\\'
file_train_data = 'training_data.csv'
file_train_label = 'training_label.csv'
file_test_data = 'test_data.csv'

In [4]:
# READ IN - ALSO RESTARTING POINT BEFORE MASSAGING, FETURES, DUMMY CODES
df_train_data = pd.read_csv(workdir + file_train_data)
df_train_label = pd.read_csv(workdir + file_train_label)
df_test_data = pd.read_csv(workdir + file_test_data)

In [5]:
# Merge the train data with the ID
df_train_data = pd.merge(df_train_data, df_train_label, on ='id')

In [6]:
# COLUMN DATA TYPE MASSAGING
#  TODO: If time permits - find a more elgant way to code

#------------------
# DATE RECORDED
#------------------
df_train_data['date_recorded'] = df_train_data['date_recorded'].astype('datetime64[ns]')
df_test_data['date_recorded'] = df_test_data['date_recorded'].astype('datetime64[ns]')

#------------------
# CONSTRUCTION YEAR
#------------------
# Convert to a string - Had issues with keeping as a int (converting over to float)
# not the cleanest work-around, but works
df_train_data['construction_year'] = df_train_data['construction_year'].astype(str)

# Set this to June 1 (chnged to 1-1 becuase of negative vlues)
# TODO: Go back and make this more intelligent.
df_train_data.loc[df_train_data['construction_year'] != '0','construction_year'] = (df_train_data['construction_year'] + '-01-01') #.astype('datetime64[ns]')

# if equal to 0 set to np.nan
df_train_data.loc[df_train_data['construction_year'] == '0','construction_year'] = ''

# convert to datetime
df_train_data['construction_year'] = df_train_data['construction_year'].astype('datetime64[ns]')

# Test data
df_test_data['construction_year'] = df_test_data['construction_year'].astype(str)
df_test_data.loc[df_test_data['construction_year'] != '0','construction_year'] = (df_test_data['construction_year'] + '-01-01') #.astype('datetime64[ns]')
df_test_data.loc[df_test_data['construction_year'] == '0','construction_year'] = ''
df_test_data['construction_year'] = df_test_data['construction_year'].astype('datetime64[ns]')



## FEATURE ENGINEERING

In [7]:
#####################
# FEATURE ENGINEERING - Run first before dummy codes
#####################
from datetime import datetime

# function to determine how old the well is (months)
def func_ft_diff_month(cols):
    d1 = cols[0]
    d2 = cols[1]
    
    if d1 == np.datetime64('NaT') or d2 == np.datetime64('NaT'):
        return 0
    else:
        return (d1.year - d2.year) * 12 + d1.month - d2.month

# Build feature
df_train_data['ft_monthsSinceBuild'] = df_train_data[['date_recorded','construction_year']].apply(func_ft_diff_month,axis=1)
df_test_data['ft_monthsSinceBuild'] = df_test_data[['date_recorded','construction_year']].apply(func_ft_diff_month,axis=1)

# Averge the rest?? - 34% -- Perhaps break this out into additional features
df_train_data[df_train_data['ft_monthsSinceBuild']==np.NaN]['ft_monthsSinceBuild'] = df_train_data['ft_monthsSinceBuild'].mean()
df_test_data[df_test_data['ft_monthsSinceBuild']==np.NaN]['ft_monthsSinceBuild'] = df_test_data['ft_monthsSinceBuild'].mean()


# Fill in the months since the build with an average (or other)
# when time permits- split this out into a feature
train_ftMnths_mean = df_train_data['ft_monthsSinceBuild'].mean()
test_ftMnths_mean = df_test_data['ft_monthsSinceBuild'].mean()

df_train_data['ft_monthsSinceBuild'].replace({np.nan: train_ftMnths_mean}, inplace=True)
df_test_data['ft_monthsSinceBuild'].replace({np.nan: test_ftMnths_mean}, inplace=True)
# or replace with 12 or 0 (at least one year or nothing)
#df_train_data['ft_monthsSinceBuild'].replace({np.nan: 12}, inplace=True)
#df_test_data['ft_monthsSinceBuild'].replace({np.nan: 12}, inplace=True)


In [8]:
# Bucket the Population sizes
def func_ft_PopSizeEst(cols):
    popSize = cols[0].astype('int64')
    
    if popSize == 0:
        return 'ft_PopSize_Unknown'
    if (popSize >= 0 and popSize < 50):
        return 'ft_PopSize_Small'
    if (popSize >= 50 and popSize < 100):
        return 'ft_PopSize_Medium'
    if popSize > 100 and popSize < 500:
        return 'ft_PopSize_Large'
    if popSize >= 500:
        return 'ft_PopSize_ExtraLarge'
    
# Population
df_train_data['ft_PopSizeEst'] = df_train_data[['population']].apply(func_ft_PopSizeEst, axis=1)
df_test_data['ft_PopSizeEst'] = df_test_data[['population']].apply(func_ft_PopSizeEst, axis=1)

# verified
df_train_data[['population','ft_PopSizeEst']].head(3)

,population,ft_PopSizeEst
0,109,ft_PopSize_Large
1,280,ft_PopSize_Large
2,250,ft_PopSize_Large


In [9]:
df_train_data.tail()
#df_test_data[df_test_data['ft_monthsSinceBuild']==np.NaN]['ft_monthsSinceBuild']

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group,ft_monthsSinceBuild,ft_PopSizeEst
59395,60739,10.0,2013-05-03,Germany Republi,1210,CES,37.169807,-3.253847,Area Three Namba 27,0,...,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional,172.000000,ft_PopSize_Large
59396,27263,4700.0,2011-05-07,Cefa-njombe,1212,Cefa,35.249991,-9.070629,Kwa Yahona Kuvala,0,...,enough,enough,river,river/lake,surface,communal standpipe,communal standpipe,functional,184.000000,ft_PopSize_Medium
59397,37057,0.0,2011-04-11,NaN,0,NaN,34.017087,-8.750434,Mashine,0,...,enough,enough,machine dbh,borehole,groundwater,hand pump,hand pump,functional,186.533251,ft_PopSize_Unknown
59398,31282,0.0,2011-03-08,Malec,0,Musa,35.861315,-6.378573,Mshoro,0,...,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump,functional,186.533251,ft_PopSize_Unknown
59399,26348,0.0,2011-03-23,World Bank,191,World,38.104048,-6.747464,Kwa Mzee Lugawa,0,...,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump,functional,110.000000,ft_PopSize_Large


## CREATE DUMMY CODES

**!! The List variables in this section control which variables we are using !! **

In [10]:
# do each one in a consolidated
def convert_dummy_vars(dataFrame, columnName, dropFirst=False, colRename=''):
    
    temp = pd.get_dummies(dataFrame[columnName], drop_first=dropFirst)
    
    if colRename != '':
        temp.columns = [(colRename)]
    
    dataFrame.drop([columnName], axis=1, inplace=True)
    dataFrame = pd.concat([dataFrame, temp], axis=1)    
    return dataFrame

In [11]:
# CHOOSE WHICH FEATURES TO INCLUDE
# 'amount_tsh', - leave out - add after binning and plotting
shortList_v1 = ['public_meeting','quantity','payment','waterpoint_type','source_class']
shortList_v2 = ['public_meeting','quantity','payment','waterpoint_type','source_class','ft_PopSizeEst','ft_monthsSinceBuild']

# Need to creat columns to OMIT

# gps_height
LongList_v1 = ['basin','district_code','public_meeting','scheme_management','construction_year',
                'extraction_type_class','management','payment','quantity','source','waterpoint_type','source_class',
             'ft_monthsSinceBuild','ft_PopSizeEst']

columnsToMaybeTry = ('region')

# Choose which feature list we are working with
# shortList_v1 - did the best so far

featureColList = 'shortList_v2'

# Carve off a new feature dataframe
df_feat_train = df_train_data[shortList_v2]
df_feat_test = df_test_data[shortList_v2]

In [12]:
# Template
#for column in df:
#    print(df[column])

# Next loop through all columns to convert to dummy variables
for column in df_feat_train:
    
    if column == 'ft_monthsSinceBuild':
        # do nothing - skip
        a = ''
    
    elif column == 'public_meeting':
        # special case
        df_feat_train = convert_dummy_vars(dataFrame=df_feat_train, columnName='public_meeting', dropFirst=True, colRename='public_meeting_True')
    else:
        df_feat_train = convert_dummy_vars(df_feat_train, column)
    
    print(df_feat_train.shape)
    
print('\n')
print('Test Shapes')
    
for column in df_feat_test:
    
    if column == 'ft_monthsSinceBuild':
        # do nothing - skip
        a = ''
    
    elif column == 'public_meeting':
        # special case
        df_feat_test = convert_dummy_vars(dataFrame=df_feat_test, columnName='public_meeting', dropFirst=True, colRename='public_meeting_True')
    else:
        df_feat_test = convert_dummy_vars(df_feat_test, column)


(59400, 7)
(59400, 11)
(59400, 17)
(59400, 23)
(59400, 25)
(59400, 29)
(59400, 29)


Test Shapes


c:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [13]:
df_feat_train.head(5)

,ft_monthsSinceBuild,public_meeting_True,dry,enough,insufficient,seasonal,unknown,never pay,other,pay annually,...,improved spring,other,groundwater,surface,unknown,ft_PopSize_ExtraLarge,ft_PopSize_Large,ft_PopSize_Medium,ft_PopSize_Small,ft_PopSize_Unknown
0,146.000000,1,0,1,0,0,0,0,0,1,...,0,0,1,0,0,0,1,0,0,0
1,38.000000,0,0,0,1,0,0,1,0,0,...,0,0,0,1,0,0,1,0,0,0
2,49.000000,1,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
3,324.000000,1,1,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,1,0,0
4,186.533251,1,0,0,0,1,0,1,0,0,...,0,0,0,1,0,0,0,0,0,1


In [14]:
df_feat_test.head(5)

,ft_monthsSinceBuild,public_meeting_True,dry,enough,insufficient,seasonal,unknown,never pay,other,pay annually,...,improved spring,other,groundwater,surface,unknown,ft_PopSize_ExtraLarge,ft_PopSize_Large,ft_PopSize_Medium,ft_PopSize_Small,ft_PopSize_Unknown
0,13.0,1,0,0,0,1,0,1,0,0,...,0,1,0,1,0,0,1,0,0,0
1,157.0,1,0,0,1,0,0,1,0,0,...,0,0,1,0,0,0,1,0,0,0
2,37.0,1,0,0,1,0,0,1,0,0,...,0,1,0,1,0,1,0,0,0,0
3,312.0,0,1,0,0,0,0,0,0,0,...,0,1,1,0,0,0,1,0,0,0
4,158.0,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0


In [15]:
# df_feat_train.rename(columns={"True":'public_meeting'}, inplace=True)

In [16]:
# df_feat_train.head()

In [17]:
####################
# create dummy codes
####################
# df_feat_train = pd.concat([df_feat_train, pd.get_dummies(df_feat_train['public_meeting'], drop_first=True)], axis=1)
# df_feat_train.rename(columns={'True':'public_meeting'}, inplace=True)
# df_feat_train.head()

In [18]:

'''
# First Iteration
df_feat_train = convert_dummy_vars(df_feat_train, 'quantity')
#df_feat_train = convert_dummy_vars(dataFrame=df_feat_train, columnName='public_meeting')
# Because F comes before T (false / true) - renaming accordingly
df_feat_train = convert_dummy_vars(dataFrame=df_feat_train, columnName='public_meeting', dropFirst=True, colRename='public_meeting_True')

df_feat_train = convert_dummy_vars(dataFrame=df_feat_train, columnName='payment')
df_feat_train = convert_dummy_vars(dataFrame=df_feat_train, columnName='waterpoint_type')
df_feat_train = convert_dummy_vars(dataFrame=df_feat_train, columnName='source_class')



# SECOND ITERATION
# Determine which avenues to take
if featureColList == 'shortList_v1':
    df_feat_train = pd.concat([df_feat_train, pd.get_dummies(df_feat_train['public_meeting'])], axis=1)
    df_feat_train.rename(columns={'True':'public_meeting'}, inplace=True)
    
    df_feat_train = pd.concat([df_feat_train, pd.get_dummies(df_feat_train['quantity'])], axis=1)
    

df_feat_train = pd.concat([df_feat_train, pd.get_dummies(df_feat_train['quantity'])], axis=1)
df_feat_train = pd.concat([df_feat_train, pd.get_dummies(df_feat_train['basin'])], axis=1)
#df_feat_train = pd.concat([df_feat_train, pd.get_dummies(df_feat_train['gps_height'])], axis=1)
df_feat_train = pd.concat([df_feat_train, pd.get_dummies(df_feat_train['district_code'])], axis=1)
df_feat_train = pd.concat([df_feat_train, pd.get_dummies(df_feat_train['public_meeting'])], axis=1)
df_feat_train.rename(columns={'True':'public_meeting'}, inplace=True)
df_feat_train = pd.concat([df_feat_train, pd.get_dummies(df_feat_train['scheme_management'])], axis=1)
#df_feat_train = pd.concat([df_feat_train, pd.get_dummies(df_feat_train['construction_year'])], axis=1)
df_feat_train = pd.concat([df_feat_train, pd.get_dummies(df_feat_train['extraction_type_class'])], axis=1)
df_feat_train = pd.concat([df_feat_train, pd.get_dummies(df_feat_train['management'])], axis=1)
df_feat_train = pd.concat([df_feat_train, pd.get_dummies(df_feat_train['payment'])], axis=1)
df_feat_train = pd.concat([df_feat_train, pd.get_dummies(df_feat_train['source'])], axis=1)
df_feat_train = pd.concat([df_feat_train, pd.get_dummies(df_feat_train['waterpoint_type'])], axis=1)
df_feat_train = pd.concat([df_feat_train, pd.get_dummies(df_feat_train['source_class'])], axis=1)

df_feat_train.drop(LongerList, axis=1, inplace=True)

   
df_feat_test = pd.concat([df_feat_test, pd.get_dummies(df_feat_test['quantity'])], axis=1)
df_feat_test = pd.concat([df_feat_test, pd.get_dummies(df_feat_test['basin'])], axis=1)
#df_feat_test = pd.concat([df_feat_test, pd.get_dummies(df_feat_test['gps_height'])], axis=1)
df_feat_test = pd.concat([df_feat_test, pd.get_dummies(df_feat_test['district_code'])], axis=1)
df_feat_test = pd.concat([df_feat_test, pd.get_dummies(df_feat_test['public_meeting'])], axis=1)
df_feat_test.rename(columns={'True':'public_meeting'}, inplace=True)
df_feat_test = pd.concat([df_feat_test, pd.get_dummies(df_feat_test['scheme_management'])], axis=1)
#df_feat_test = pd.concat([df_feat_test, pd.get_dummies(df_feat_test['construction_year'])], axis=1)
df_feat_test = pd.concat([df_feat_test, pd.get_dummies(df_feat_test['extraction_type_class'])], axis=1)
df_feat_test = pd.concat([df_feat_test, pd.get_dummies(df_feat_test['management'])], axis=1)
df_feat_test = pd.concat([df_feat_test, pd.get_dummies(df_feat_test['payment'])], axis=1)
df_feat_test = pd.concat([df_feat_test, pd.get_dummies(df_feat_test['source'])], axis=1)
df_feat_test = pd.concat([df_feat_test, pd.get_dummies(df_feat_test['waterpoint_type'])], axis=1)
df_feat_test = pd.concat([df_feat_test, pd.get_dummies(df_feat_test['source_class'])], axis=1)

df_feat_test.drop(LongerList, axis=1, inplace=True)
'''

"\n# First Iteration\ndf_feat_train = convert_dummy_vars(df_feat_train, 'quantity')\n#df_feat_train = convert_dummy_vars(dataFrame=df_feat_train, columnName='public_meeting')\n# Because F comes before T (false / true) - renaming accordingly\ndf_feat_train = convert_dummy_vars(dataFrame=df_feat_train, columnName='public_meeting', dropFirst=True, colRename='public_meeting_True')\n\ndf_feat_train = convert_dummy_vars(dataFrame=df_feat_train, columnName='payment')\ndf_feat_train = convert_dummy_vars(dataFrame=df_feat_train, columnName='waterpoint_type')\ndf_feat_train = convert_dummy_vars(dataFrame=df_feat_train, columnName='source_class')\n\n\n\n# SECOND ITERATION\n# Determine which avenues to take\nif featureColList == 'shortList_v1':\n    df_feat_train = pd.concat([df_feat_train, pd.get_dummies(df_feat_train['public_meeting'])], axis=1)\n    df_feat_train.rename(columns={'True':'public_meeting'}, inplace=True)\n    \n    df_feat_train = pd.concat([df_feat_train, pd.get_dummies(df_feat_t

In [19]:

df_feat_train.head(3)

,ft_monthsSinceBuild,public_meeting_True,dry,enough,insufficient,seasonal,unknown,never pay,other,pay annually,...,improved spring,other,groundwater,surface,unknown,ft_PopSize_ExtraLarge,ft_PopSize_Large,ft_PopSize_Medium,ft_PopSize_Small,ft_PopSize_Unknown
0,146.0,1,0,1,0,0,0,0,0,1,...,0,0,1,0,0,0,1,0,0,0
1,38.0,0,0,0,1,0,0,1,0,0,...,0,0,0,1,0,0,1,0,0,0
2,49.0,1,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0


In [20]:
# Create dummy variables
'''

public_meeting = pd.get_dummies(df_feat_train['public_meeting'], drop_first=True)
quantity = pd.get_dummies(df_feat_train['quantity'])
payment = pd.get_dummies(df_feat_train['payment'])
waterpoint_type = pd.get_dummies(df_feat_train['waterpoint_type'])
source_class = pd.get_dummies(df_feat_train['source_class'])

# rename this
public_meeting.columns = ['public_meeting']


test_public_meeting = pd.get_dummies(df_feat_test['public_meeting'], drop_first=True)
test_quantity = pd.get_dummies(df_feat_test['quantity'])
test_payment = pd.get_dummies(df_feat_test['payment'])
test_waterpoint_type = pd.get_dummies(df_feat_test['waterpoint_type'])
test_source_class = pd.get_dummies(df_feat_test['source_class'])

# rename this
test_public_meeting.columns = ['public_meeting']
'''

"\n\npublic_meeting = pd.get_dummies(df_feat_train['public_meeting'], drop_first=True)\nquantity = pd.get_dummies(df_feat_train['quantity'])\npayment = pd.get_dummies(df_feat_train['payment'])\nwaterpoint_type = pd.get_dummies(df_feat_train['waterpoint_type'])\nsource_class = pd.get_dummies(df_feat_train['source_class'])\n\n# rename this\npublic_meeting.columns = ['public_meeting']\n\n\ntest_public_meeting = pd.get_dummies(df_feat_test['public_meeting'], drop_first=True)\ntest_quantity = pd.get_dummies(df_feat_test['quantity'])\ntest_payment = pd.get_dummies(df_feat_test['payment'])\ntest_waterpoint_type = pd.get_dummies(df_feat_test['waterpoint_type'])\ntest_source_class = pd.get_dummies(df_feat_test['source_class'])\n\n# rename this\ntest_public_meeting.columns = ['public_meeting']\n"

In [21]:
# drop the existing columns
# df_feat_train.drop(['public_meeting','quantity','payment','waterpoint_type','source_class'], axis=1, inplace=True)
# df_feat_train = pd.concat([df_feat_train,public_meeting, quantity, payment, waterpoint_type, source_class], axis=1)

# df_feat_test.drop(['public_meeting','quantity','payment','waterpoint_type','source_class'], axis=1, inplace=True)
# df_feat_test = pd.concat([df_feat_test,test_public_meeting, test_quantity, test_payment, test_waterpoint_type, test_source_class], axis=1)


In [22]:
df_feat_train.tail(5)

,ft_monthsSinceBuild,public_meeting_True,dry,enough,insufficient,seasonal,unknown,never pay,other,pay annually,...,improved spring,other,groundwater,surface,unknown,ft_PopSize_ExtraLarge,ft_PopSize_Large,ft_PopSize_Medium,ft_PopSize_Small,ft_PopSize_Unknown
59395,172.000000,1,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
59396,184.000000,1,0,1,0,0,0,0,0,1,...,0,0,0,1,0,0,0,1,0,0
59397,186.533251,1,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
59398,186.533251,1,0,0,1,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,1
59399,110.000000,1,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0


In [23]:
df_feat_test.tail(5)

,ft_monthsSinceBuild,public_meeting_True,dry,enough,insufficient,seasonal,unknown,never pay,other,pay annually,...,improved spring,other,groundwater,surface,unknown,ft_PopSize_ExtraLarge,ft_PopSize_Large,ft_PopSize_Medium,ft_PopSize_Small,ft_PopSize_Unknown
14845,277.0,1,0,1,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,1,0
14846,206.0,1,0,0,1,0,0,0,0,1,...,0,0,1,0,0,1,0,0,0,0
14847,38.0,1,0,0,1,0,0,1,0,0,...,0,0,0,1,0,0,1,0,0,0
14848,49.0,1,0,0,1,0,0,1,0,0,...,0,0,0,1,0,0,1,0,0,0
14849,61.0,1,1,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,1,0


## SVM MODEL

In [ ]:
# Use a SVM 

In [ ]:
##############
# LOCAL SPLIT
##############

X = df_feat_train
y = df_train_label['status_group']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35, random_state=101)

model = SVC()
model.fit(X_train, y_train)
predictions = model.predict(X_test)

In [ ]:
# Print Predictions
print(confusion_matrix(y_test, predictions))
print('\n')
print(classification_report(y_test, predictions))

In [ ]:
######################################
# let's try with a different parameter
######################################
bParamSearch = 0

if (bParamSearch == 1):
    param_grid = {'C':[0.1,1,10,100,1000], 'gamma':[1,0.1,0.01,0.001,0.0001]}
    grid = GridSearchCV(SVC(), param_grid, verbose=3)
    grid.fit(X_train, y_train)
    
    # best parameter
    # {'C': 0.1, 'gamma': 1}
    grid.best_params_
    
    grid.best_estimator_
    grid_predictions = grid.predict(X_test)

In [ ]:
# Recheck everything
if (bParamSearch == 1):
    print(confusion_matrix(y_test, grid_predictions))
    print('\n')
    print(classification_report(y_test, grid_predictions))

In [24]:
####################################
# WITH FULL RUN 
####################################

# X = df_feat_train
# y = df_train_label['status_group']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35, random_state=101)

X_train = df_feat_train
y_train = df_train_label['status_group']
X_test = df_feat_test

In [25]:
y_train.head(3)

0    functional
1    functional
2    functional
Name: status_group, dtype: object

In [26]:
# Find the best parameter - if we flag to do so
bFindProdParmeter = 0
if bFindProdParmeter == 1:
    param_grid = {'C':[0.1,1,10,100,1000], 'gamma':[1,0.1,0.01,0.001,0.0001]}
    grid = GridSearchCV(SVC(), param_grid, verbose=3,n_jobs=6)
    grid.fit(X_train, y_train)

In [27]:
# if we are seraching for best parameter, then print it (to keep the verbose logging above separate)
if bFindProdParmeter == 1:
    print(grid.best_params_)
    # {'C': 0.1, 'gamma': 1}
    # {'gamma': 0.1, 'C': 10}  - shortlist2
    print(grid.best_estimator_)

In [28]:
# shortlist
# {'C': 0.1, 'gamma': 1}
# .7162

# shortlist2
# {'gamma': 0.1, 'C': 10}  

# start tracking Time
print('Starting Model Run')
start_time = time.time()

###################################
# RUN THE MODEL
model = SVC(C=10, gamma=0.1)
model.fit(X_train, y_train)
predictions = model.predict(X_test)
###################################

elapsed_time = time.time() - start_time
if elapsed_time > 60:
    print('Finish Load Scaled Train Data: '+str(elapsed_time/60) + ' min and ' + str(elapsed_time/(60*2)) + ' seconds' )
else:
    print('Finish Load Scaled Train Data: '+str(elapsed_time))

In [29]:
# Now merge the predictions with the id
pd.DataFrame(predictions).head(3)

,0
0,non functional
1,functional
2,functional


In [30]:
df_predictions = pd.DataFrame(predictions)
df_predictions.columns=['status_group']

In [31]:
df_predictions.head(3)

,status_group
0,non functional
1,functional
2,functional


In [32]:
df_test_data.head(3)

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,ft_monthsSinceBuild,ft_PopSizeEst
0,50785,0.0,2013-02-04,Dmdd,1996,DMDD,35.290799,-4.059696,Dinamu Secondary School,0,...,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,other,other,13.0,ft_PopSize_Large
1,51630,0.0,2013-02-04,Government Of Tanzania,1569,DWE,36.656709,-3.309214,Kimnyak,0,...,good,insufficient,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe,157.0,ft_PopSize_Large
2,17168,0.0,2013-02-01,NaN,1567,NaN,34.767863,-5.004344,Puma Secondary,0,...,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,other,other,37.0,ft_PopSize_ExtraLarge


In [33]:
# TODO: Need to add a date/time stamp and record what each model does
# df_final = pd.merge(df_predictions, df_test_data, on='index')
df_final = pd.concat([df_predictions, df_test_data], axis=1)[['id','status_group']]
df_final.set_index('id', inplace=True)
export_file = workdir + 'submission.csv'
df_final.to_csv(export_file)